## Tutorial: Stitching Networks between Chameleon Sites

This notebook demonstrates how to use ExoGENI to stitch networks between Chameleon sites. It will walk through reserving and configuring stitchable networks on each Chameleon site and using the xoStitch tool to create a layer2 circuit between the networks. 

## Tutorial: 

#### Configure the Environment

Import the chi example API calls, set the project name and region, and set various names and attributes to use in the tutorial. 

In [ ]:
import json
import os
import chi

#Config with your project and site
chi.set('project_name', 'chameleon')   # Replace with your project name
chi.use_site('CHI@UC')                 # Authenticate to CHI@UC site

#GENI Pem file
geni_pem_file = '/home/pruth/work/geni-pruth.pem'

# Tip: Name resources with your username for easier identification
username = os.getenv("USER")
prefix = username + "_StitchingTutorial_"
server_name = prefix+'Server'
network_name = prefix+'Net'
subnet_name = prefix+'Subnet'
router_name = prefix+'Router'
lease_name = prefix+'Lease'


#Server Config
image_name='CC-Ubuntu20.04'
node_type="compute_cascadelake_r"
server_count=1

#Network Config
#physical_network='exogeni'
physical_network='physnet1'
cidr='192.168.42.0/24'

## Create Chicago Network and Server

#### Create a Lease at Chicago

In [ ]:
import chi.lease
from datetime import datetime, timedelta
from dateutil import tz

BLAZAR_TIME_FORMAT = '%Y-%m-%d %H:%M'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
chi.lease.add_node_reservation(reservation_list, count=server_count, node_type=node_type)
chi.lease.add_network_reservation(reservation_list, network_name=network_name, physical_network=physical_network)
chi.lease.add_fip_reservation(reservation_list, count=1)

# Create the lease
uc_lease = chi.lease.create_lease(lease_name,
                                  reservations=reservation_list,
                                  start_date=start_date,
                                  end_date=end_date)
    
#Print the lease info
print(json.dumps(lease, indent=2))

#### Get the Reservations

Each lease contains one or more reservations. The individual reservation IDs are required to instantiate resources. You can [get the lease](../modules-python/reservations/get_lease_by_name.ipynb) and separate the reservation IDs for compute, network, and floating IPs using the technique below.

In [ ]:
uc_compute_reservation_id = [reservation for reservation in uc_lease['reservations'] if reservation['resource_type'] == 'physical:host'][0]['id']
uc_network_reservation_id = [reservation for reservation in uc_lease['reservations'] if reservation['resource_type'] == 'network'][0]['id']
uc_floatingip_reservation_id = [reservation for reservation in uc_lease['reservations'] if reservation['resource_type'] == 'virtual:floatingip'][0]['id']

print(f"uc_compute_reservation_id: {uc_compute_reservation_id}")
print(f"uc_network_reservation_id: {uc_network_reservation_id}")
print(f"uc_floatingip_reservation_id: {uc_floatingip_reservation_id}")

#### Get the Network

Getting the network is not required for the remainder of the tutorial. However, it is a good test to see if your network reservation has become active. The [get_network_by_name](../modules-python/network/get_network_by_name.ipynb) call will fail if a network with that name does not yet exits. It will also fail if a network with the same name already exists (likely from a previous run of this notebook).

In [ ]:
import chi.network

#Get the network
uc_network = chi.network.get_network(network_name)
print(json.dumps(uc_network, indent=2))

#Get the network ID
uc_network_id = uc_network['id']
print(f'Network ID: {uc_network_id}')

#Get the VLAN tag (needed for ExoGENI stitching)
uc_network_vlan = uc_network['provider:segmentation_id']
print(f'uc_network_vlan: {uc_network_vlan}')

#### Add a subnet to the reserved network

In [ ]:
uc_subnet = chi.network.create_subnet(subnet_name, uc_network_id, cidr=cidr)

print(json.dumps(uc_subnet, indent=2))

#### Add a Router

In [ ]:
uc_router = chi.network.create_router(router_name, gw_network_name='public')

print(json.dumps(uc_router, indent=2))

#### Attach the Router and Subnet

In [ ]:
chi.network.add_subnet_to_router_by_name(router_name, subnet_name)

## Create TACC Network and Server

#### Create a Lease at TACC

In [ ]:
chi.set('region_name', 'CHI@TACC')     # Switch regions to TACC
chi.use_site('CHI@TACC')               # Authenticate to CHI@TACC site

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
chi.lease.add_node_reservation(reservation_list, count=server_count, node_type=node_type)
chi.lease.add_network_reservation(reservation_list, network_name=network_name, physical_network=physical_network)
chi.lease.add_fip_reservation(reservation_list, count=1)

# Create the lease
tacc_lease = chi.lease.create_lease(lease_name,
                                    reservations=reservation_list,
                                    start_date=start_date,
                                    end_date=end_date)
    
#Print the lease info
print(json.dumps(tacc_lease, indent=2))

#### Get the Reservations

Each lease contains one or more reservations. The individual reservation IDs are required to instantiate resources. You can [get the lease](../modules-python/reservations/get_lease_by_name.ipynb) and separate the reservation IDs for compute, network, and floating IPs using the technique below.

In [ ]:
tacc_compute_reservation_id = [reservation for reservation in tacc_lease['reservations'] if reservation['resource_type'] == 'physical:host'][0]['id']
tacc_network_reservation_id = [reservation for reservation in tacc_lease['reservations'] if reservation['resource_type'] == 'network'][0]['id']
tacc_floatingip_reservation_id = [reservation for reservation in tacc_lease['reservations'] if reservation['resource_type'] == 'virtual:floatingip'][0]['id']

print(f"tacc_compute_reservation_id: {tacc_compute_reservation_id}")
print(f"tacc_network_reservation_id: {tacc_network_reservation_id}")
print(f"tacc_floatingip_reservation_id: {tacc_floatingip_reservation_id}")

#### Get the Network

Getting the network is not required for the remainder of the tutorial. However, it is a good test to see if your network reservation has become active. The `get_network` call will fail if a network with that name does not yet exist. It will also fail if a network with the same name already exists (likely from a previous run of this notebook).

In [ ]:
import chi.network

#Get the network
tacc_network = chi.network.get_network(network_name)
print(json.dumps(tacc_network, indent=2))

#Get the network ID
tacc_network_id = tacc_network['id']
print(f'Network ID: {tacc_network_id}')

#Get the VLAN tag (needed for ExoGENI stitching)
tacc_network_vlan = tacc_network['provider:segmentation_id']
print(f'tacc_network_vlan: {tacc_network_vlan}')

#### Add a subnet to the reserved network

In [ ]:
tacc_subnet = chi.network.create_subnet(subnet_name, tacc_network_id, cidr=cidr)

print(json.dumps(tacc_subnet, indent=2))

#### Add a Router

In [ ]:
tacc_router = chi.network.create_router(router_name, gw_network_name='public')

print(json.dumps(tacc_router, indent=2))

#### Attach the Router and Subnet

In [ ]:
chi.network.add_subnet_to_router_by_name(router_name, subnet_name)

## Start the Servers

#### Start the UC Server

Use the compute_reservation_id to create the server.

In [ ]:
chi.set('region_name', 'CHI@UC')
chi.use_site('CHI@UC')
import chi.server

# Create the server
uc_server = chi.server.create_server(server_name, 
                                     reservation_id=uc_compute_reservation_id, 
                                     network_name=network_name, 
                                  image_name=image_name)
# Wait until the server is active
chi.server.wait_for_active(uc_server.id)

#### Associate the Floating IP

In [ ]:
floating_ip = chi.server.associate_floating_ip(uc_server.id)

print(f'Floating IP: {floating_ip}')

#### Start the TACC Server

Use the compute_reservation_id to create the server.

In [ ]:
chi.set('region_name', 'CHI@TACC')
chi.use_site('CHI@TACC')

# Create the server
tacc_server = chi.server.create_server(server_name, 
                                  reservation_id=tacc_compute_reservation_id, 
                                  network_name=network_name, 
                                  image_name=image_name)
# Wait until the server is active
chi.server.wait_for_active(tacc_server.id)

#### Associate the Floating IP

In [ ]:
floating_ip = chi.server.associate_floating_ip(tacc_server.id)

print(f'Floating IP: {floating_ip}')

## Stitch the Circuit using ExoGENI

Note: The ExoGENI
steps require a valid GENI certificate at the path specified and a public/private keypair in ~/.ssh (run ssh-keygen with default inputs)

#### Create the Circuit

In [ ]:
%%script env uc_vlan="$uc_network_vlan" tacc_vlan="$tacc_network_vlan" geni_pem="$geni_pem_file" bash

echo 'uc_vlan ' $uc_vlan ', tacc_vlan ' $tacc_vlan ', geni_pem ' $geni_pem
xoStitch create -sp1 uc -vlan1 $uc_vlan -sp2 tacc -vlan2 $tacc_vlan -c $geni_pem

#### Check the Status of the Circuit

In [ ]:
%%script env uc_vlan="$uc_network_vlan" tacc_vlan="$tacc_network_vlan" geni_pem="$geni_pem_file" bash

echo 'uc_vlan ' $uc_vlan ', tacc_vlan ' $tacc_vlan ', geni_pem ' $geni_pem
xoStitch status -sp1 uc -vlan1 $uc_vlan -sp2 tacc -vlan2 $tacc_vlan -c $geni_pem

## Clean Up Resources

### Delete Stitched Circuit using ExoGENI

In [ ]:
%%script env uc_vlan="$uc_network_vlan" tacc_vlan="$tacc_network_vlan" geni_pem="$geni_pem_file" bash

echo 'uc_vlan ' $uc_vlan ', tacc_vlan ' $tacc_vlan ', geni_pem ' $geni_pem
xoStitch delete -sp1 uc -vlan1 $uc_vlan -sp2 tacc -vlan2 $tacc_vlan -c $geni_pem

### Delete TACC Resources

Delete the server

In [ ]:
chi.set('region_name', 'CHI@TACC')
chi.use_site('CHI@TACC')

In [ ]:
chi.server.delete_server(tacc_server.id)

#### De-configure Network

In [ ]:
def de_configure_network(router, subnet, network_id):
    router_id = router['id']
    subnet_id = subnet['id']

    try:
        result = chi.network.remove_subnet_from_router(router_id, subnet_id)
    except Exception as e:
        print(f"detach_router_by_name error: {str(e)}")
        pass

    try:
        result = chi.network.delete_router(router_id)
    except Exception as e:
        print(f"delete_router_by_name error: {str(e)}")
        pass

    try:
        result = chi.network.delete_subnet(subnet_id)
    except Exception as e:
        print(f"delete_subnet_by_name error: {str(e)}")
        pass

    try:
        result = chi.network.delete_network(network_id)
    except Exception as e:
        print(f"delete_network_by_name error: {str(e)}")
        pass

In [ ]:
de_configure_network(tacc_router, tacc_subnet, tacc_network_id)

#### Release Lease

In [ ]:
chi.lease.delete_lease(tacc_lease['id'])

### Delete UC Resources

Delete the server.

In [ ]:
chi.set('region_name', 'CHI@UC')
chi.use_site('CHI@UC')

In [ ]:
chi.server.delete_server(uc_server.id)

#### De-configure Network

In [ ]:
de_configure_network(uc_router, uc_subnet, uc_network_id)

#### Release Lease

In [ ]:
chi.lease.delete_lease(uc_lease['id'])